In [ ]:
# Не удаляйте эту ячейку!

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

data = pd.read_csv('/datasets/flights_preprocessed.csv')

target = data['Arrival Delay']
features = data.drop(['Arrival Delay'] , axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [5]:
# median
model = LinearRegression()
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)

print(mean_absolute_error(target_valid, predictions_valid))

27.436250978085834


In [14]:
%%time

best_score, deep = 100, 0
for depth in range(1, 16, 1):
    model = RandomForestRegressor(n_estimators=20, max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    score_1 = mean_absolute_error(target_valid, predictions_valid)
#     print(score_1, depth, sep="\n")
    if best_score > score_1:
        best_score = score_1
        deep = depth

print(best_score, deep)# < напишите код здесь >

25.543812554450305 12
CPU times: user 27.5 s, sys: 136 ms, total: 27.6 s
Wall time: 27.7 s


In [16]:
%%time

model = RandomForestRegressor(n_estimators=100, 
    max_depth = 12, random_state=12345)
model.fit(features_train, target_train)# обучите модель на тренировочной выборке

predictions_train = model.predict(features_train)# получите предсказания на обучающей выборке
predictions_valid = model.predict(features_valid)# получите предсказания на валидационной выборке

print(mean_absolute_error(target_train, predictions_train))# посчитайте mean_absolute_error на обучающей выборке
print(mean_absolute_error(target_valid, predictions_valid))# посчитайте mean_absolute_error на валидационной выборке

22.469041005849554
25.47745895064548
CPU times: user 14.1 s, sys: 116 ms, total: 14.2 s
Wall time: 14.2 s
